# RNNLM

*Recurrent Neural Network Language Model*

RNNを用いて言語モデルを作成する。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
from typing import List

import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')


---

## データセット

wiki40b

In [3]:
with tf.device('/cpu:0'):
    ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

In [4]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [5]:
data = data[:500]


---

## 前処理

やることは2章と同じ。実装方法を若干変えている。

### トークン化

In [6]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
def tokenize(text: str) -> List[str]:
    return tagger.parse(text).strip().split()

for sentence in data:
    tokens = tokenize(sentence)
    data_tokens.append(tokens)
data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### 語彙の学習

特殊トークンを含めた語彙を学習させる。`torchtext.vocab.Vocab`オブジェクトの作成。

In [7]:
bos, eos, unk = '<bos>', '<eos>', '<unk>'
specials = [bos, eos, unk]
vocab = build_vocab_from_iterator(data_tokens, specials=specials)
unk_id = vocab[unk]
vocab.set_default_index(unk_id)
n_vocab = len(vocab)
print('num of vocabrary:', n_vocab)

num of vocabrary: 14491


### 前処理まとめ

In [8]:
transform = Compose([
    transforms.AddToken(bos, begin=True), # 先頭に<BOS>を追加
    transforms.AddToken(eos, begin=False), # 末尾に<EOS>を追加
    transforms.VocabTransform(vocab), # 単語をIDに変換
    transforms.ToTensor() # Tensorに変換
])
data_ids = [transform(tokens) for tokens in data_tokens]
data_ids[0] # example

tensor([    0,    21,  6187,  1046,     5,     9,  6319, 13959,    52,    22,
           94,    54,     3,  3377,    36,  8120,     7, 11268,    34,     5,
         3249,    14,     4,    94,    54,     3,  7390,     7,  4451,    24,
            6, 12246,   145,    11,    14,    10,  1597,    34,     5,   111,
           18,    17,    10,    86,     4, 11684,     3,  3080,    82,     5,
          573,   111,    18,    43,     3,    12,  6085,    11,    39,    10,
           25,     6,     1])


---

## 学習データ

入力と出力のペアを作成する。  
では、どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNLMは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。この3つの単語の誤差とその勾配は一度に求められる。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。

では学習データを作成しよう。PyTorchのDataLoaderのような、入力と正解のペアがtupleで取り出せるイテレータを作成する。  
なおバッチサイズは1とする。ミニバッチへの対応は後程。

In [9]:
class TextLoader:
    def __init__(self, data_ids):
        self.data = data_ids
        self.n_data = len(data_ids)

    def __iter__(self):
        random.shuffle(self.data)
        for text in self.data:
            yield text[:-1], text[1:]

    def __len__(self):
        return self.n_data

dataloader = TextLoader(data_ids)
sample_y, sample_t = next(iter(dataloader))
sample_y[:20], sample_t[:20] # example

(tensor([   0, 2758,   15,   47,   31,  526,   37,    5, 2436,   16, 8552,   11,
         2041,  331,    7,  888,   83,    6, 1482,    7]),
 tensor([2758,   15,   47,   31,  526,   37,    5, 2436,   16, 8552,   11, 2041,
          331,    7,  888,   83,    6, 1482,    7, 1778]))


---

## モデル構築

埋め込み層、RNN層、線形層から構築する。

In [10]:
class RNNLM(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        x = self.embedding(x) # (seq_len, embed_size)
        y, h = self.rnn(x, h) # y: (seq_len, hidden_size)
        y = self.fc(y) # (seq_len, n_vocab)
        return y, h

モデル内部のRNN層へ隠れ状態を入力できるようにしている。また、RNN層から出力された隠れ状態を受け取れるようにしている。これらは推論時に再帰的な処理を書けるようにするため。


---

## 学習

学習は普通。特に変わったことはしていない。  
モデルが出力した単語列と正解の単語列の誤差を交差エントロピーで求めて以下略。

In [13]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            x, t = x.to(device), t.to(device)
            optimizer.zero_grad()
            y, _ = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [16]:
model = RNNLM(n_vocab, 1024, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [17]:
train(model, optimizer, 100, 10)

   1-10/100: ######################################## 100% [00:01:59.40] loss: 4.31548 
  11-20/100: ######################################## 100% [00:01:59.55] loss: 2.02370 
  21-30/100: ######################################## 100% [00:01:57.53] loss: 0.88471 
  31-40/100: ######################################## 100% [00:01:56.18] loss: 0.36398 
  41-50/100: ######################################## 100% [00:01:55.75] loss: 0.17184 
  51-60/100: ######################################## 100% [00:01:57.76] loss: 0.10994 
  61-70/100: ######################################## 100% [00:01:56.78] loss: 0.09935 
  71-80/100: ######################################## 100% [00:01:58.26] loss: 0.09392 
  81-90/100: ######################################## 100% [00:01:57.25] loss: 0.08936 
 91-100/100: ######################################## 100% [00:01:57.39] loss: 0.09239 



---

## 文章生成

学習したモデルを使って文章を生成する。

In [18]:
def token_sampling(y):
    y = y[-1]
    probs = F.softmax(y, dim=-1)
    token = random.choices(range(n_vocab), weights=probs)[0]
    return token


@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()
    start_tokens = [bos] + tokenize(start)
    start_ids = vocab(start_tokens)
    start_ids = torch.tensor(start_ids, device=device)

    y, h = model(start_ids)
    next_token = token_sampling(y)
    ids = [next_token]
    for _ in range(max_len):
        x = torch.tensor([next_token], device=device)
        y, h = model(x, h)
        next_token = token_sampling(y)
        ids.append(next_token)
        if next_token == vocab[eos]:
            break
    tokens = [vocab.get_itos()[t] for t in ids[:-1]]
    return start + ''.join(tokens)

In [23]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

ゲーム内の「タイムアタッククエスト」をアレンジしたステージをどれだけ早くクリアできるか競う勝ち抜き戦。各予選会場で勝ち上がった上位2チームが決勝大会に進出する。アークスグランプリが開始された当時はPC版
平均点は県内私立の矢板東中学校にも匹敵する。
「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再
一般に、「歴史学の父」といわれるヘロドトスは紀元前5世紀のギリシャの人である。彼らは音楽業界のことを何も知らず、エンターテイメントとも対立のことを提案した。
名詞同様に形態的な単数・複数の区別や声調による主格と対格の区別がある。形容詞が名詞を修飾する場合は名詞の後ろに置かれる。
この時代のイギリスやフランスで主流となった功利主義・進化論・実証主義の共通する特徴は、内的要因よりも外的要因を重視することであるとも考えられ、その後間、キリスト教徒の
わずかな時間だけ表示される数枚のコインを見て、表示されたコインの総額を答える。
「ファンタシースター感謝祭」の前身である。島代であるため、2006年（明治40年）2月に勲四等に叙せられ、旭日小綬章を受けた。同年6月に逝去。享年64
細胞膜上に散らばって存在するNa⁺-K⁺ポンプにより、K⁺の細胞内への能動輸送とNa⁺の細胞外への能動輸送が行われる。このポンプは、三つのNa⁺を細胞
坂上宝剣は『公衡公記(昭訓門院御にすでに先行する研究があり、ここではその1つの拠点として整備された。小規模宅地と密集する改宗にローマた元老院を


In [25]:
generate_sentence(model, start='昨日の夜、')

'昨日の夜、スウィフトのゴアはテネシー州の選挙区から下院議会選挙に出馬し、当選した。3回当選を重ねた後の1984年には下院に出馬せず、上院議会選挙に立候補して当選し、'

まあまあまともな文章が生成されるようになった気がする。


---

## Truncated BPTT

RNNは時間ごとに隠れ状態を出力する。学習時は時間を逆にたどって逆伝播を行う。  
この時間を跨いだ逆伝播はBPTT（*Backpropagation Through Time*）とも呼ばれる。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなるが、そもそもRNNの時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

では実装していこう。といっても、学習部分をちょっと変えるだけ。

In [26]:
def train(model, optimizer, n_epochs, trunc_len, prog_unit=1): # trunc_len: 区切る長さ
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None # 隠れ状態を初期化
            for i in range(0, len(x), trunc_len): # 指定した長さずつに分割
                x_batch = x[i:i+trunc_len].to(device) # バッチを作成
                t_batch = t[i:i+trunc_len].to(device) # バッチを作成
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y, t_batch)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 隠れ状態を計算グラフから切り離す
            prog.update(advance=1)

変更した行にはコメントを記述した

In [27]:
model = RNNLM(n_vocab, 1024, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [28]:
train(model, optimizer, 50, 64, 5)

  1-5/50: ######################################## 100% [00:02:16.99] loss: 4.58593 
 6-10/50: ######################################## 100% [00:02:18.03] loss: 2.50928 
11-15/50: ######################################## 100% [00:02:18.18] loss: 1.17610 
16-20/50: ######################################## 100% [00:02:18.70] loss: 0.45621 
21-25/50: ######################################## 100% [00:02:18.02] loss: 0.17406 
26-30/50: ######################################## 100% [00:02:18.28] loss: 0.09264 
31-35/50: ######################################## 100% [00:02:19.24] loss: 0.07003 
36-40/50: ######################################## 100% [00:02:19.97] loss: 0.06355 
41-45/50: ######################################## 100% [00:02:19.28] loss: 0.05992 
46-50/50: ######################################## 100% [00:02:19.16] loss: 0.05729 


In [31]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

近代歴史学は文化史・唯物論歴史学という全く異なる方向性を追求する歴史学へと発展したが、一方でそれらとは別個に歴史研究における構想力を重視し、幅広い要求
30:𨰽
1909年に個人商店として神田駿河台に設立され、1920年に株式会社に改組し、現在の社名となった。富山県砺波市に工場を持つ。主な製品として、「セレクト」
2015年はアークスキャラバンおよびアークスフェスティバル2015が開催され、夏季にファンタシースター感謝祭は行われなかった。しかし「アークスキャラバン」実施中のPSO2放送局内でプロデューサー・酒井智史が2015年度内に開催する予定
林が1998年に出版した『主婦の復権』（講談社）ISBN9784062091947をめぐって、「わいふ」前編集長・田中喜美子との間で多くの論争があった。林はラディカル・フェミニズムの家族
2008年1月12日の放送分からセットが変更され、山瀬以外の席順や福留・久保の座り位置（2007年までは画面左が福留、画面右が久保）が入れ替わる。同年7月26日
梵名のハヤグリーヴァは「馬の首」の意である。これはヒンドゥー教では最高神ヴィシュヌの異名でもあり、馬頭観音の成立におけるその影響が指摘されている。他にも「馬頭
アフリカ大陸でのロケ撮影において、ヨルダンのペトラ遺跡とエジプトのピラミッドをロケ地にした撮影が行われた。、ただ背景として映しただけである、コウモリの定着率が生まれ、その
6人の人物がボウリング（1人1投のみ）をするので、合計21本になりそうな人物の組合せを予想する。1人目と2人目のボウリングで倒したピンの合計数から、
人女性はスウィフトとは別にドメスティクス伯（Comesdomesticorum）によって率いられる皇帝護衛担当の親衛隊（Domesticus）もあった。この部隊は特別の任務につき、その構成員は将来の士官候補生



---

## ミニバッチ学習

先程はバッチサイズ1で学習を行ったが、やはりミニバッチでないと効率が悪いので、ミニバッチ学習を行う。

言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [32]:
from torch.nn.utils.rnn import pad_sequence

In [33]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。

In [34]:
# パディングトークンの追加
PAD = '<pad>'
vocab.append_token(PAD)
pad_id = vocab.get_stoi()[PAD]
n_vocab = len(vocab)

class TextDataset(Dataset):
    def __init__(self, text_data, transform):
        self._n_samples = len(text_data)
        self.data = [transform(text) for text in text_data]

    def __getitem__(self, index):
        in_text = self.data[index][:-1]
        out_text = self.data[index][1:]
        return in_text, out_text

    def __len__(self):
        return self._n_samples

def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

dataset = TextDataset(data_tokens, transform)
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
sample = next(iter(dataloader))
sample[0].shape

torch.Size([32, 939])

学習部分も少し変更点がある。  
損失を計算する際に、パディング用のトークンを無視するようにする。  
その他実装上の変更はコメントを参照。

In [35]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # padトークンを無視
def train(model, optimizer, n_epochs, trunc_len, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None
            x = x.to(device)
            t = t.to(device)
            for i in range(0, len(x), trunc_len):
                x_batch = x[:, i:i+trunc_len] # 軸を変更
                t_batch = t[:, i:i+trunc_len] #    "
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y.reshape(-1, n_vocab), t_batch.ravel())
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach_()
            prog.update(advance=1)

In [39]:
model = RNNLM(n_vocab, 1024, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

では学習を行う。ミニバッチによって1epohにかかる時間が短くなるため、epoch数を増やす。

In [40]:
n_epochs = 2000
train(model, optimizer, n_epochs, 64, prog_unit=200)

    1-200/2000: ######################################## 100% [00:01:38.02] loss: 3.52016 
  201-400/2000: ######################################## 100% [00:01:38.79] loss: 2.65203 
  401-600/2000: ######################################## 100% [00:01:38.63] loss: 2.43914 
  601-800/2000: ######################################## 100% [00:01:37.46] loss: 2.19903 
 801-1000/2000: ######################################## 100% [00:01:34.60] loss: 1.98344 
1001-1200/2000: ######################################## 100% [00:01:35.33] loss: 1.81617 
1201-1400/2000: ######################################## 100% [00:01:35.04] loss: 1.69927 
1401-1600/2000: ######################################## 100% [00:01:34.81] loss: 1.61504 
1601-1800/2000: ######################################## 100% [00:01:34.92] loss: 1.55808 
1801-2000/2000: ######################################## 100% [00:01:34.89] loss: 1.51957 


In [41]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

いくつのパイロット、エジプトなどの子としてきた。開幕をする啓蒙主義や、頭部に加わるとドイツ関税同盟が運行された。1908年までの歴史意識の関係、より弁別が設置・レヴェル
PSVitaである。これは日本円、中である。また満州電信を突破、2008年（馬のジャンルに選出された。1980年）のは4月27日の頭で昇格にまた、2年
承平年中（トランスフォーマーシリーズのおば応期生の名称は別個にある物がコンスタンティヌス1マスに入っては、食べることを主張しては膜上については、南インドの車両が弱まり、ロースクールを
最初の二代中期は、4色にフランス革命後、イオンチャネルの隠居に発売さを記録とリキニウスの子とPS2回行い、神経細胞の終盤戦に308年7月には、またローマ帝国が
その物と連動した。高校卒業後の「テイラー・進化論歴史研究があったアルゼンチン代表であった。活動を称し、同チャートで、同じく、初期の出陣式が着実な方法論・テーズ直々に
下呂市久保尼寺小路（Xacitarxanまたは、ドイツが、デビッド・ダイアはボスポラス海峡にアナウンサーは、国分寺市、ジメシチルジセレニド触媒量の一つの歴史学の一流品のアイテムが工業化して、アニメーション制作プロデューサーの
報道の大迫害に優勝した。学校図書館」、1830年になった。
巻き毛の大迫害の最も偉大な事実をしているの寄進状によりイランにより、ジャズ・インスツルメンツ、電位のかを称し、初期の影響を探す（彼から歴史研究が、体色は跨線橋で更に
1995-竹篙厝-1面2回NHK紅白歌合戦』（現神戸市にアナウンサーはソリドゥス金貨の一種）で、所謂エピソードから大金を発する。特に西方の都市ビュザンティオン（1982年に3月には、
懐徳JCT-1億5日には、作者は寝技の時に受かり、ドイツが技術は2)は憲法により1983年（下益城郡宇奈月町（下益城郡が技術提携日に開催された平和の編集


lossが思ったように下がらなかった。生成された文章も微妙だね。バッチサイズは小さい方がいいのかもね。